In [1]:
import alinea.adel.mtg as mtg

In [2]:
import alinea.adel.fitting as fitting

In [3]:
# import alinea.adel.parameterisation.parameterisation as adelp

In [4]:
from alinea.adel.symbol import build_symbols

In [5]:
from alinea.adel.data_samples import leaves_db as ldb

In [6]:
from openalea.mtg.traversal import pre_order2

In [7]:
from openalea.plantgl.all import Viewer  

In [18]:
from cereals import build_shoot, parametric_leaf, leaf_azimuth
from simple_maize import bell_shaped_dist, geometric_dist

In [15]:
import numpy as np

In [9]:
g = None

def adel_mtg():

    # generation of a 3D plant from descritive parameters
    stem_radius=0.5 # realistic, number or growing with phytomer age ?
    height=2000 # from crop model
    nb_phy=15 # from 0 to fixed max nb of phytomers
    max_leaf_length=2000
    insertion_angle=50
    scurv=0.5
    curvature=50
    phyllotactic_angle=120
    spiral=True
    
    insertion_heights=np.array(geometric_dist(height, nb_phy, q=1.2)) # further separate stem and pseudo stem, cf simple maize
    
    leaf_lengths=np.array(bell_shaped_dist(plant_area=max_leaf_length, nb_phy=nb_phy, rmax=0.7, skew=0.15)) # plant area --> max leaf length
    # leaf_areas=bell_shaped_dist(plant_area=1, nb_phy=15, rmax=0.7, skew=0.15) # cf blade_dimension
    
    a_leaf = parametric_leaf(nb_segment=10, insertion_angle=insertion_angle, scurv=scurv, curvature=curvature, alpha=-2.3)
    leaf_shapes = [a_leaf for l in leaf_lengths] # replace leaf_length by nb_phy or...
    
    leaf_azimuths = leaf_azimuth(size=len(leaf_lengths), phyllotactic_angle=phyllotactic_angle, phyllotactic_deviation=15, plant_orientation=0, spiral=spiral)
    
    shoot, g = build_shoot(stem_radius=stem_radius, insertion_heights=insertion_heights, leaf_lengths=leaf_lengths, leaf_areas=None,
                    leaf_shapes=leaf_shapes, leaf_azimuths=leaf_azimuths)

    


    return g

In [10]:
def leaves_db():
    leaves = ldb()
    leaves = fitting.fit_leaves(leaves, 9)
    
    db = leaves[0]
    functions = build_symbols(db)
    return functions

In [11]:
def test_adel_mtg():
    g = adel_mtg()
    #print g
    symbols = leaves_db()

    assert len(g) == len(g.sub_mtg(g.root))

    print(list(symbols.keys()))
    g=mtg.mtg_turtle(g,symbols)
    return g

In [45]:
def test_dynamic_mtg():
    """ Add to the each metamers a thermal time (e.g. 10).
    Length = length * (thermal_time - start_thermaltime)/(end-start)
    if the thermal time is lower that the start thermal time, 
    do not proceed more.
    """

    g = adel_mtg()
    g.display()

    v = next(g.component_roots_at_scale_iter(g.root, scale=1))
    tt = 0
    dtt = 10.
    for metamer in pre_order2(g, v):
        nm = g.node(metamer)
        for node in nm.components():
            node.start_tt = tt
            node.end_tt = tt+dtt
        tt += dtt

    return g, tt

In [46]:
def display():

    g, max_time = test_dynamic_mtg()

    # add 1 scale --> >=3 scales
    
    # for vid in g.vertices(scale=1):
    #     g.node(vid=vid).start_tt=0
    #     g.node(vid=vid).end_tt=10
    
    symbols = leaves_db()

    for time in range(1,150):
        g.properties()['geometry'] = {}
        g = mtg.mtg_turtle_time(g, symbols, time)
        scene = g.to_plantgl()
        Viewer.display(scene)

In [47]:
display()

MTG : nb_vertices=32, nb_scales=2
/plant			(id=1)											
^<StemElement			(id=2)											
	+LeafElement			(id=3)										
^<StemElement			(id=4)											
	+LeafElement			(id=5)										
^<StemElement			(id=6)											
	+LeafElement			(id=7)										
^<StemElement			(id=8)											
	+LeafElement			(id=9)										
^<StemElement			(id=10)											
	+LeafElement			(id=11)										
^<StemElement			(id=12)											
	+LeafElement			(id=13)										
^<StemElement			(id=14)											
	+LeafElement			(id=15)										
^<StemElement			(id=16)											
	+LeafElement			(id=17)										
^<StemElement			(id=18)											
	+LeafElement			(id=19)										
^<StemElement			(id=20)											
	+LeafElement			(id=21)										
^<StemElement			(id=22)											
	+LeafElement			(id=23)										
^<StemElement			(id=24)											
	+LeafElement			(id=25)										
^<StemElement			(id=26)											
	+LeafElement			(id=27)										
^<StemElement			(id=28)											
	+LeafElement		

StopIteration: 